## optunaをインストール

In [89]:
!pip install optuna

In [90]:
#optunaをimport
#optunaはlightGBMのハイパーパラメータをチューニングするライブラリ
import optuna

## データ読み込み

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

#出力する行数を設定、省略をなくす
pd.set_option('display.max_rows', 500)

#データをまとめて読み込み、訓練データはtrain
train_files = glob.glob('./data/train/*.csv')
train = pd.DataFrame()
for file in train_files:
    train1 = pd.read_csv(file)
    cat_train = pd.concat([train,train1])
    train = cat_train

train.head()

/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_94259/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_94259/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_94259/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_94259/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)


,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,40020197,中古マンション等,NaN,40133,福岡県,福岡市中央区,谷,桜坂,7,２ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,150.0,2021年第2四半期,改装済,NaN,7.041393
1,40031380,中古マンション等,NaN,40137,福岡県,福岡市早良区,西新,西新,4,１Ｋ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第2四半期,未改装,NaN,6.602060
2,40030436,中古マンション等,NaN,40137,福岡県,福岡市早良区,小田部,室見,18,４ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,150.0,2020年第4四半期,改装済,NaN,7.397940
3,40015295,中古マンション等,NaN,40132,福岡県,福岡市博多区,竹丘町,雑餉隈,5,２ＬＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第1四半期,未改装,NaN,7.278754
4,40147771,中古マンション等,NaN,40133,福岡県,福岡市中央区,荒戸,大濠公園,6,３ＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2007年第3四半期,未改装,NaN,6.929419


In [3]:
train.columns

Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')

In [4]:
#目的変数
"""
'取引価格（総額）_log'
"""

#説明変数
"""
'ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
        '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）', '改装'
"""

###欠損値の削除###
class Delete_Null_Data():
    """
    欠損率が高いcolumnを削除するclass
    """
    
    def __init__(self,df):
        self.df = df
        
    #nullのカウントとnullの%のDataframeを返すメソッド
    def null_counts(self):
        null_counts_df = self.df
        null_counts_df = pd.DataFrame(null_counts_df.isnull().sum(),columns=['null_counts'])
        null_counts_df['null_par'] = null_counts_df['null_counts']/len(self.df)
        return null_counts_df
    
    #nullの多いcolumnを削除するメソッド引数にnullの%を指定　１００％→１　５０％→0.5
    def delete_null_columns(self,null_par=0.9):
        df = self.null_counts()
        re_df = df[df['null_par']>null_par]
        cols = re_df.index.tolist()
        delete_df = self.df.drop(labels=cols,axis=1)
        
        return delete_df

#インスタンス化して訓練データを更新する
Adj_Data=Delete_Null_Data(train)
train=Adj_Data.delete_null_columns()
train.shape

(722574, 20)

In [5]:
###カテゴリ変数化###
for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装"]:
                train[col] = train[col].astype("category")

In [6]:
train.columns

Index(['ID', '種類', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称', '最寄駅：距離（分）',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引価格（総額）_log'],
      dtype='object')

In [7]:
###'最寄駅：距離（分）'###
#時間がわからない部分の補完、欠損値は10で補完
train['最寄駅：距離（分）']=train['最寄駅：距離（分）'].replace({'30分?60分':'45', '1H30?2H':'105', '1H?1H30':'75', '2H?':'120'})
train['最寄駅：距離（分）']=train['最寄駅：距離（分）'].fillna('10')

#文字列型から整数型に変換
train['最寄駅：距離（分）']=train['最寄駅：距離（分）'].astype(int)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722574 entries, 0 to 37378
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   ID            722574 non-null  int64   
 1   種類            722574 non-null  object  
 2   市区町村コード       722574 non-null  int64   
 3   都道府県名         722574 non-null  category
 4   市区町村名         722574 non-null  object  
 5   地区名           721906 non-null  category
 6   最寄駅：名称        719843 non-null  category
 7   最寄駅：距離（分）     722574 non-null  int64   
 8   間取り           696682 non-null  category
 9   面積（㎡）         722574 non-null  object  
 10  建築年           701593 non-null  object  
 11  建物の構造         703422 non-null  category
 12  用途            651713 non-null  category
 13  今後の利用目的       356654 non-null  category
 14  都市計画          702756 non-null  category
 15  建ぺい率（％）       698413 non-null  float64 
 16  容積率（％）        698413 non-null  float64 
 17  取引時点          722574 non-null 

In [9]:
###面積###
#面積の欠損値はなし、'2000㎡以上'は'2000'で補完
train['面積（㎡）']=train['面積（㎡）'].replace('2000㎡以上','2000')

#文字列型から整数型に変換
train['面積（㎡）']=train['面積（㎡）'].astype(int)

In [10]:
###建築年###
#欠損値を補完, nanのままだとうまくいかないので文字列でnullと入れておく
train['建築年']=train['建築年'].fillna('null')

#和暦から西暦に変換する
import random

def seireki(df):
    l=[]

    y=df['建築年']
    
    for i in range(0,len(y),1):
    #for i in range(0,10):
        
        #年号が令和、令和2年=2018+2=2020年
        if y.iloc[i][:2]=='令和':
            if y.iloc[i][3]=='年':
                year=2022-(2018+int(y.iloc[i][2]))
            else:
                year=2022-(2018+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l.append(year)

        
        #年号が平成、平成2年=1988+2=1991年
        elif y.iloc[i][:2]=='平成':
            if y.iloc[i][3]=='年':
                year=2022-(1988+int(y.iloc[i][2]))
            else:
                year=2022-(1988+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l.append(year)
            

        #年号が昭和、昭和2年=1925+2=1927年
        elif y.iloc[i][:2]=='昭和':
            if y.iloc[i][3]=='年':
                year=2022-(1925+int(y.iloc[i][2]))
            else:
                year=2022-(1925+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l.append(year)

        #年号が戦前、1868年から1945年の値をランダムに格納
        #1945年に終戦
        elif y.iloc[i][:2]=='戦前':
            year=2022-random.randint(1868, 1945)
            l.append(year)

        #年号が欠損、1868年から2021年の値をランダムに格納
        else:
            year=2022-random.randint(1868, 2021)
            l.append(year)
    
    
    #戻り値：西暦に変換した値が格納されたリスト
    return l

#trainデータをseireki関数の引数として渡して、結果を変数resultに格納
result=seireki(train)
print(len(result))

#resultをtrain['建築年']に格納
train['建築年']=result

722574


In [11]:
###取引時点###
#2014年第4四半期

#年号+時期
def torihiki(df):
    l=[]

    y=df['取引時点']
    
    for i in range(0,len(y),1):
    #for i in range(0,10):
        
        #第1四半期
        if y.iloc[i][6]=='1':
            torihiki=int(y.iloc[i][:4])+0.25
            l.append(torihiki)

        #第2四半期
        elif y.iloc[i][6]=='2':
            torihiki=int(y.iloc[i][:4])+0.50
            l.append(torihiki)
            
        #第3四半期
        elif y.iloc[i][6]=='2':
            torihiki=int(y.iloc[i][:4])+0.50
            l.append(torihiki)
            
        #第4四半期
        else:
            torihiki=int(y.iloc[i][:4])+0.99
            l.append(torihiki)
            
    return l    


In [12]:
#変換した数値をtrain['取引時点']に格納
train['取引時点']=torihiki(train)

In [13]:
train.columns

Index(['ID', '種類', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称', '最寄駅：距離（分）',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引価格（総額）_log'],
      dtype='object')

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722574 entries, 0 to 37378
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   ID            722574 non-null  int64   
 1   種類            722574 non-null  object  
 2   市区町村コード       722574 non-null  int64   
 3   都道府県名         722574 non-null  category
 4   市区町村名         722574 non-null  object  
 5   地区名           721906 non-null  category
 6   最寄駅：名称        719843 non-null  category
 7   最寄駅：距離（分）     722574 non-null  int64   
 8   間取り           696682 non-null  category
 9   面積（㎡）         722574 non-null  int64   
 10  建築年           722574 non-null  int64   
 11  建物の構造         703422 non-null  category
 12  用途            651713 non-null  category
 13  今後の利用目的       356654 non-null  category
 14  都市計画          702756 non-null  category
 15  建ぺい率（％）       698413 non-null  float64 
 16  容積率（％）        698413 non-null  float64 
 17  取引時点          722574 non-null 

In [15]:
train.head(10)

,ID,種類,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引価格（総額）_log
0,40020197,中古マンション等,40133,福岡県,福岡市中央区,谷,桜坂,7,２ＬＤＫ,45,34,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,150.0,2021.50,改装済,7.041393
1,40031380,中古マンション等,40137,福岡県,福岡市早良区,西新,西新,4,１Ｋ,15,33,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2020.50,未改装,6.602060
2,40030436,中古マンション等,40137,福岡県,福岡市早良区,小田部,室見,18,４ＬＤＫ,80,34,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,150.0,2020.99,改装済,7.397940
3,40015295,中古マンション等,40132,福岡県,福岡市博多区,竹丘町,雑餉隈,5,２ＬＤＫ,60,16,ＲＣ,住宅,住宅,商業地域,80.0,400.0,2020.25,未改装,7.278754
4,40147771,中古マンション等,40133,福岡県,福岡市中央区,荒戸,大濠公園,6,３ＤＫ,60,35,ＳＲＣ,住宅,NaN,商業地域,80.0,400.0,2007.99,未改装,6.929419
5,40113879,中古マンション等,40134,福岡県,福岡市南区,向新町,大橋(福岡),22,３ＬＤＫ,75,20,ＲＣ,住宅,NaN,第２種住居地域,60.0,200.0,2013.99,未改装,7.301030
6,40088445,中古マンション等,40133,福岡県,福岡市中央区,小笹,桜坂,18,３ＬＤＫ,70,27,ＲＣ,住宅,住宅,第１種中高層住居専用地域,60.0,150.0,2015.25,NaN,7.113943
7,40145262,中古マンション等,40131,福岡県,福岡市東区,香住ヶ丘,九産大前,10,４ＬＤＫ,75,21,ＳＲＣ,住宅,NaN,NaN,NaN,NaN,2006.50,未改装,7.255273
8,40090009,中古マンション等,40203,福岡県,久留米市,善導寺町,善導寺,8,３ＬＤＫ,70,25,ＲＣ,住宅,NaN,第１種住居地域,60.0,200.0,2010.25,未改装,6.886491
9,40015036,中古マンション等,40132,福岡県,福岡市博多区,住吉,博多,16,１ＬＤＫ,40,35,ＳＲＣ,住宅,住宅,近隣商業地域,80.0,300.0,2020.99,未改装,6.869232


In [16]:
###種類, 市区町村名, 取引時点は削除###
train=train.drop(['種類','市区町村名'], axis=1)

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722574 entries, 0 to 37378
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   ID            722574 non-null  int64   
 1   市区町村コード       722574 non-null  int64   
 2   都道府県名         722574 non-null  category
 3   地区名           721906 non-null  category
 4   最寄駅：名称        719843 non-null  category
 5   最寄駅：距離（分）     722574 non-null  int64   
 6   間取り           696682 non-null  category
 7   面積（㎡）         722574 non-null  int64   
 8   建築年           722574 non-null  int64   
 9   建物の構造         703422 non-null  category
 10  用途            651713 non-null  category
 11  今後の利用目的       356654 non-null  category
 12  都市計画          702756 non-null  category
 13  建ぺい率（％）       698413 non-null  float64 
 14  容積率（％）        698413 non-null  float64 
 15  取引時点          722574 non-null  float64 
 16  改装            654498 non-null  category
 17  取引価格（総額）_log  722574 non-null 

## モデルの学習（訓練データ）

In [18]:
#説明変数と目的変数に分割
train_X=train.drop('取引価格（総額）_log', axis=1)
train_y=train['取引価格（総額）_log']

In [19]:
train_X.columns, train_y.head()

(Index(['ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
        '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時点',
        '改装'],
       dtype='object'),
 0    7.041393
 1    6.602060
 2    7.397940
 3    7.278754
 4    6.929419
 Name: 取引価格（総額）_log, dtype: float64)

In [20]:
#重回帰分析
#ランダムフォレスト
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

#訓練検証データとテストデータに分割
trainval_X, test_X, trainval_y, test_y = train_test_split(train_X, train_y, test_size=0.3, random_state=1)

#訓練データと検証データに分割
train_X, val_X, train_y, val_y = train_test_split(trainval_X, trainval_y, test_size=0.2, random_state=1)



In [21]:
train_X.head()

,ID,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装
1672,7008516,7201,福島県,野田町,福島(福島),9,３ＬＤＫ,75,17,ＲＣ,住宅,住宅,第１種住居地域,60.0,200.0,2015.25,未改装
9811,13217077,13119,東京都,大山西町,大山(東京),7,３ＬＤＫ,65,4,ＲＣ,NaN,住宅,第１種中高層住居専用地域,60.0,200.0,2020.50,未改装
11854,13028657,13103,東京都,芝浦,日の出,6,NaN,25,15,ＲＣ,住宅,NaN,商業地域,80.0,400.0,2011.99,未改装
17986,28063606,28214,兵庫県,伊孑志,逆瀬川,8,３ＬＤＫ,65,49,ＲＣ,住宅,住宅,第１種住居地域,60.0,200.0,2019.25,改装済
16345,14010057,14103,神奈川県,南浅間町,西横浜,6,１Ｋ,15,36,NaN,NaN,住宅,商業地域,80.0,400.0,2017.99,未改装


In [30]:
import lightgbm as lgb

trains = lgb.Dataset(train_X, train_y)
valids = lgb.Dataset(val_X, val_y)

params = {
    'objective': 'regression',
    'metrics': 'mae'
    
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=5000, early_stopping_rounds=100,verbose_eval=1000)

/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026466 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12012
[LightGBM] [Info] Number of data points in the train set: 404640, number of used features: 17
[LightGBM] [Info] Start training from score 7.227810


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's l1: 0.0799449
Early stopping, best iteration is:
[1161]	valid_0's l1: 0.0798867


In [31]:
###学習データの予測結果###
pred=model.predict(train_X)
print("MAE: ",np.mean(np.abs(train_y-pred)))

"""
#文字化け防止
import matplotlib.pyplot as plt
plt.rc('font', family='Hiragino sans')

#各特徴量の重要度を確認
lgb.plot_importance(model, figsize=(12,8), max_num_features=50, importance_type='gain')

"""


MAE:  0.0671723956880263


"\n#文字化け防止\nimport matplotlib.pyplot as plt\nplt.rc('font', family='Hiragino sans')\n\n#各特徴量の重要度を確認\nlgb.plot_importance(model, figsize=(12,8), max_num_features=50, importance_type='gain')\n\n"

In [33]:
model.params

{'objective': 'regression',
 'metrics': 'mae',
 'num_iterations': 5000,
 'early_stopping_round': 100,
 'categorical_column': [2, 3, 4, 6, 9, 10, 11, 12, 16]}

In [32]:
###学習データの予測結果###
pred=model.predict(val_X)
print("MAE: ",np.mean(np.abs(val_y-pred)))

MAE:  0.07988666513409544


## テストデータ読み込み、前処理

In [41]:
#目的変数
"""
'取引価格（総額）_log'
"""

#説明変数
"""
'ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
        '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）','取引時点', '改装'
"""

"\n'ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',\n        '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）','取引時点', '改装'\n"

In [42]:
#テストデータ読み込み
test=pd.read_csv('./data/test.csv')
#人口データ読み込み
#population_test=pd.read_csv('population_test.csv')
#提出用サンプルファイル読み込み
sample=pd.read_csv('./data/sample_submission.csv')

#欠損値の多いカラムを除去
#インスタンス化して訓練データを更新する
Adj_Data=Delete_Null_Data(test)
test=Adj_Data.delete_null_columns()

test.head()

,ID,種類,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装
0,1000078,中古マンション等,1101,北海道,札幌市中央区,大通西,西１１丁目,1,３ＬＤＫ,75,平成28年,ＲＣ,住宅,住宅,商業地域,80.0,600.0,2021年第3四半期,未改装
1,1000121,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,0,１ＤＫ,30,昭和52年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第3四半期,改装済
2,1000123,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,3,３ＬＤＫ,70,昭和59年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第3四半期,改装済
3,1000127,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,2,２ＬＤＫ,50,昭和64年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第4四半期,未改装
4,1000129,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１ＤＫ,45,平成3年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第4四半期,改装済


In [43]:
#必要なカラムだけ抽出したのち、前処理
test=test.drop(['種類','市区町村名'], axis=1)
test.columns

Index(['ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
       '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時点',
       '改装'],
      dtype='object')

In [44]:
###前処理###
###カテゴリ変数化###
for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装"]:
                test[col] = test[col].astype("category")

In [45]:
###'最寄駅：距離（分）'###
#時間がわからない部分の補完、欠損値は10で補完
test['最寄駅：距離（分）']=test['最寄駅：距離（分）'].replace({'30分?60分':'45', '1H30?2H':'105', '1H?1H30':'75', '2H?':'120'})
test['最寄駅：距離（分）']=test['最寄駅：距離（分）'].fillna('10')

#文字列型から整数型に変換
test['最寄駅：距離（分）']=test['最寄駅：距離（分）'].astype(int)

In [46]:
###面積###
#面積の欠損値はなし、'2000㎡以上'は'2000'で補完
test['面積（㎡）']=test['面積（㎡）'].replace('2000㎡以上','2000')

#文字列型から整数型に変換
test['面積（㎡）']=test['面積（㎡）'].astype(int)

In [47]:
#testデータをseireki関数の引数として渡して、結果を変数resultに格納
test['建築年']=test['建築年'].fillna('null')
result=seireki(test)
print(len(result))

#resultをtrain['建築年']に格納
test['建築年']=result

21005


In [48]:
#変換した数値をtest['取引時点']に格納
test['取引時点']=torihiki(test)

## テストデータで予測

In [50]:
###学習データの予測結果###
pred=model.predict(test)

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=pred
sample.to_csv('submission10.csv', index=None)


In [161]:
train.isnull().sum()

ID                   0
市区町村コード              0
都道府県名                0
地区名                668
最寄駅：名称            2731
最寄駅：距離（分）            0
間取り              25892
面積（㎡）                0
建築年                  0
建物の構造            19152
用途               70861
今後の利用目的         365920
都市計画             19818
建ぺい率（％）          24161
容積率（％）           24161
取引時点                 0
改装               68076
取引価格（総額）_log         0
dtype: int64